In [14]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
# go to the first page of job post
url = 'https://apply.deloitte.com/careers/SearchJobs?sort=relevancy'
driver = webdriver.Chrome()

driver.implicitly_wait(2)
driver.get(url)

In [4]:
# load all jobs to the current page by clicking the load more button
next_button = driver.find_element('xpath', '//*[@class="button button--default button--loadmore"]')
while next_button:
    driver.execute_script("arguments[0].click();", next_button)
    time.sleep(2)
    try:
        next_button = driver.find_element('xpath', '//*[@class="button button--default button--loadmore"]')
    except:
        break

In [5]:
# scrape all job titles and links
job_title=[]
job_link=[]

soup = BeautifulSoup(driver.page_source, 'html.parser')
job_title.extend([td.text for td in soup.find_all("a", {"class": "link"})])
job_link.extend([td['href'] for td in soup.find_all("a", {"class": "link"})])  

In [6]:
def remove_space(s):
  s_new = s.replace('\n', ' ').replace('\xa0', ' ')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

In [7]:
# remove unwanted chars
job_title_cleaned = [remove_space(s) for s in job_title]
job_link_cleaned = [remove_space(s) for s in job_link]

# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title_cleaned, job_link_cleaned), columns=['JOB_TITLE', 'JOB_LINK'])

# drop the duplicates
df_title_link = df_title_link.drop_duplicates()

In [8]:
len(job_title)

2360

## Extract job descriptions and qualifications for each job

In [31]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []

    qual_pattern = re.compile("qualifications|required:", re.IGNORECASE)
    descrp_pattern = re.compile("work you’ll do|job duties", re.IGNORECASE)

    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        URL=link[i]
        driver.get(URL)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        s = ''
        d = ''
        
        # get descriptions
        try:
          tag = soup.find("strong", text=descrp_pattern).findNext("ul")
          d = tag.text
        except: pass
    
        # get qualifications
        try:
          tag = soup.find(re.compile("(strong|span)"), text=qual_pattern).findNext("ul")
          s = tag.text
          if tag.findNext("ul"):
            s = s + " " + tag.findNext('ul').text
        except: pass
                      
        qualifications.append(s)
        description.append(d)   
        jobtitle.append(title[i])
        joblink.append(link[i])
        
    driver.quit()            
 
    return jobtitle, link, qualifications, description

In [40]:
# retrieve the qualification for each job.
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)

In [41]:
qual_cleaned = [remove_space(q) for q in qual]
descrp_cleaned = [remove_space(d) for d in descrp]

In [42]:
#create a dataframe from basic qualification and preferred qualification
df = pd.DataFrame(zip(title, qual_cleaned, link, descrp_cleaned))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'Deloitte'
df = df.iloc[:, [4, 0, 1, 2, 3]]

In [43]:
# look for empty QUALIFICATION entries
def get_empty(df, colname) :
  empty_idx=[]
  for i in range(len(df[colname])):
    count = len(df[colname][i])
    if count<=10 : empty_idx.append(i)
    else: pass
  return empty_idx

empty_mini_q = get_empty(df, 'QUALIFICATIONS')

# drop the empty Qualification entries
df.drop(empty_mini_q, inplace=True)

In [44]:
#remove the duplicated jobs
df_nodup = df.drop_duplicates()
print("There are {} jobs from Deloitte.".format(df_nodup.shape[0]))

df_nodup.to_csv('deloitte_all_jobs_cleaned.csv')

There are 1904 jobs from Deloitte.
